In [1]:
import os
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
from PIL import Image 
from pathlib import Path
import visualkeras
import mlflow
import mlflow.keras

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers, layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint 
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings 

warnings.filterwarnings('ignore')

In [2]:
# %%
# 1. 建立模型的函式
def build_model(learning_rate, dropout_rate, dense_units, trainable_layers, num_classes):
    """
    根據傳入的超參數建立 MobileNetV2 模型。

    參數:
    - learning_rate (float): 學習率
    - dropout_rate (float): Dropout 比率
    - dense_units (int): 全連接層的神經元數量
    - trainable_layers (int): MobileNetV2 中可訓練的層數 (從後往前算)
    - num_classes (int): 分類的類別總數
    
    返回:
    - model: 已編譯的 Keras 模型
    """
    # 建立基礎模型 MobileNetV2
    base_model = keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # 設定可訓練的層數
    base_model.trainable = True
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False
        
    # 建立序列模型
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_units, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(dense_units, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # 編譯模型
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [5]:
# ===================================================================
# 1. MLflow 視覺化與日誌記錄輔助函式
# ===================================================================

def plot_and_log_history(history, filename="history_plots.png"):
    """
    繪製準確率和損失函數的歷史曲線，並將其儲存為圖片，最後記錄到 MLflow。
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 準確率圖
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    ax1.set_ylim(0, 1)
    ax1.set_title('Accuracy over epochs')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # 損失函數圖
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title('Loss over epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    # 儲存圖片並關閉繪圖
    plt.tight_layout()
    plt.savefig(filename)
    plt.close(fig)  # 確保關閉正確的 figure
    
    # 將圖片記錄到 MLflow
    mlflow.log_artifact(filename)
    print(f"✅ '{filename}' has been logged to MLflow.")

def log_confusion_matrix(y_true, y_pred, class_labels, filename="confusion_matrix.png"):
    """ 繪製、儲存並記錄標準混淆矩陣 """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(18, 14)) # 增加圖片大小以容納更多類別
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()
    mlflow.log_artifact(filename)
    print(f"✅ '{filename}' has been logged to MLflow.")

def plot_and_log_sorted_confusion_bar(y_true, y_pred, class_labels, filename="sorted_correct_counts.png"):
    """ 計算、繪製並記錄一個長條圖，顯示每個類別的正確預測數量（由高到低排序）。 """
    cm = confusion_matrix(y_true, y_pred)
    correct_counts = np.diag(cm)
    
    sorted_idx = np.argsort(correct_counts)[::-1]
    sorted_counts = correct_counts[sorted_idx]
    sorted_labels = np.array(class_labels)[sorted_idx]

    plt.figure(figsize=(18, 8)) # 增加圖片寬度
    plt.bar(range(len(sorted_counts)), sorted_counts, color='lightgreen')
    plt.xticks(range(len(sorted_labels)), sorted_labels, rotation=90)
    plt.xlabel('Class')
    plt.ylabel('Correctly Predicted Count')
    plt.title('Correctly Predicted Count per Class (sorted)')
    plt.tight_layout()
    
    plt.savefig(filename)
    plt.close()

    mlflow.log_artifact(filename)
    print(f"✅ '{filename}' has been logged to MLflow.")

def evaluate_and_log_all_reports(model, data_generator, class_labels):
    """
    評估模型並記錄分類報告、混淆矩陣及其他分析圖到 MLflow。
    """
    print("\n" + "="*20 + " Starting Full Evaluation for Logging " + "="*20)

    # 關鍵：重置 generator 以確保順序正確
    data_generator.reset()

    # 1. 模型預測
    # 使用 predict() 方法，並指定 steps，可以確保處理完所有樣本
    predictions = model.predict(data_generator, steps=len(data_generator), verbose=1)
    y_pred = np.argmax(predictions, axis=1)

    # 2. 取得真實標籤
    # 注意：generator 的 classes 屬性包含了所有樣本的真實標籤
    y_true = data_generator.classes

    # 確保長度一致，這在 generator batch_size 無法整除總樣本數時很重要
    if len(y_pred) != len(y_true):
        print(f"⚠️ y_pred length {len(y_pred)} vs y_true length {len(y_true)}. This is normal if batch_size doesn't divide total samples.")
        # `predict` 會處理完所有樣本，所以 y_true 也應該是完整的
        y_true = y_true[:len(y_pred)]

    # 3. 產生並記錄分類報告
    report = classification_report(y_true, y_pred, target_names=class_labels, digits=4, zero_division=0)
    print("\nClassification Report:\n", report)
    mlflow.log_text(report, "classification_report.txt")
    print("✅ 'classification_report.txt' has been logged to MLflow.")

    # 4. 記錄混淆矩陣
    log_confusion_matrix(y_true, y_pred, class_labels)

    # 5. 記錄排序後的正確預測數量長條圖
    plot_and_log_sorted_confusion_bar(y_true, y_pred, class_labels)
    
    print("="*20 + " Full Evaluation Logging Complete " + "="*20 + "\n")

In [9]:
# %%
# 2. 自動訓練與調參的主函式 (已更新)
import itertools

def train_and_tune(train_gen, val_gen, class_labels, class_weight, num_classes, target_accuracy=0.9):
    """
    自動訓練和調整超參數，直到驗證準確率達到目標。
    使用 MLflow 自動記錄參數、指標、圖表和模型。
    """
    # 定義超參數的搜尋空間
    param_space = {
        'learning_rate': [0.0001],
        'dropout_rate': [0.5],
        'dense_units': [256],
        'trainable_layers': [60] # 從 MobileNetV2 的後面開始訓練的層數
    }
    
    keys, values = zip(*param_space.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    best_val_accuracy = 0
    best_model = None
    best_run_id = None

    mlflow.set_experiment("vegetable_classification_autotune")
    
    for i, params in enumerate(param_combinations):
        if best_val_accuracy >= target_accuracy:
            print(f"🎯 目標已達成 (Accuracy >= {target_accuracy:.2f})。停止搜尋。")
            break

        print("-" * 60)
        print(f"🚀 Run {i+1}/{len(param_combinations)}: 嘗試參數組合: {params}")

        with mlflow.start_run() as run:
            # 記錄超參數
            mlflow.log_params(params)
            
            # 建立並編譯模型
            model = build_model(num_classes=num_classes, **params)
            
            # 設定回呼函式
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-8)
            checkpoint = tf.keras.callbacks.ModelCheckpoint("model_checkpoint.keras", monitor="val_loss", save_best_only=True)
            
            # 模型訓練
            history = model.fit(
                train_gen,
                epochs=30,
                validation_data=val_gen,
                class_weight=class_weight,
                callbacks=[early_stop, reduce_lr, checkpoint],
                verbose=1
            )
            
            # === MLflow 記錄區 ===
            
            # 1. 記錄主要指標
            val_accuracy = max(history.history['val_accuracy'])
            mlflow.log_metric("best_val_accuracy", val_accuracy)
            mlflow.log_metric("final_train_accuracy", history.history['accuracy'][-1])
            mlflow.log_metric("stopped_epoch", len(history.history['val_accuracy']))

            # 2. 記錄訓練歷史圖表
            plot_and_log_history(history)
            
            # 3. 記錄完整的評估報告 (混淆矩陣, 分類報告等)
            # 使用訓練好的模型 (EarlyStopping 會自動還原最佳權重)
            evaluate_and_log_all_reports(model, val_gen, class_labels)
            
            # 4. 記錄模型
            mlflow.keras.log_model(model, "model", signature=None) # signature=False 可以加快儲存速度
            
            print(f"✔️ Run {run.info.run_id} 完成。最佳驗證準確率: {val_accuracy:.4f}")
            
            # 更新全局最佳模型
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_model = model
                best_run_id = run.info.run_id
                print(f"🎉 新的最佳模型! Run ID: {best_run_id}, Accuracy: {best_val_accuracy:.4f}")

    print("=" * 60)
    print("🏁 自動調參完成。")
    if best_model:
        print(f"🏆 最終最佳模型的 Run ID 為: {best_run_id}")
        print(f"🏆 最終最佳驗證準確率為: {best_val_accuracy:.4f}")
        best_model.save('best_tuned_model.h5')
        print("💾 最佳模型已儲存為 'best_tuned_model.h5'")
    else:
        print("❌ 未能成功訓練任何模型。")
        
    return best_model, best_val_accuracy

In [ ]:
# %%
# 3. 執行主流程 (已更新)

# --- 資料準備 ---
train_path = "dataset_full_en/train"
validation_path = "dataset_full_en/validation"
num_classes = len(os.listdir(train_path))

train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest'
)

train_generator_aug = train_datagen_aug.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True # 訓練集需要 shuffle
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False # 驗證/測試集絕不能 shuffle，以確保標籤順序正確
)


# 從 generator 中獲取類別名稱和索引的對應關係
class_labels = list(train_generator_aug.class_indices.keys())

# 計算 class_weight
from sklearn.utils.class_weight import compute_class_weight

class_indices = np.unique(train_generator_aug.classes)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=class_indices,
    y=train_generator_aug.classes
)
class_weight_dict = dict(zip(class_indices, class_weights_array))


# --- 啟動自動訓練與調參 ---
if __name__ == '__main__':
    # 將所有需要的參數傳入
    best_model, best_accuracy = train_and_tune(
        train_gen=train_generator_aug,
        val_gen=validation_generator,
        class_labels=class_labels,
        class_weight=class_weight_dict,
        num_classes=num_classes,
        target_accuracy=0.90
    )

Found 26307 images belonging to 60 classes.
Found 7514 images belonging to 60 classes.
------------------------------------------------------------
🚀 Run 1/1: 嘗試參數組合: {'learning_rate': 0.0001, 'dropout_rate': 0.5, 'dense_units': 256, 'trainable_layers': 60}
Epoch 1/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 540s 647ms/step - accuracy: 0.0831 - loss: 3.8904 - val_accuracy: 0.4952 - val_loss: 1.8341 - learning_rate: 1.0000e-04
Epoch 2/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 525s 638ms/step - accuracy: 0.4202 - loss: 2.1947 - val_accuracy: 0.6590 - val_loss: 1.1882 - learning_rate: 1.0000e-04
Epoch 3/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 525s 638ms/step - accuracy: 0.5720 - loss: 1.5841 - val_accuracy: 0.6928 - val_loss: 1.0851 - learning_rate: 1.0000e-04
Epoch 4/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 524s 637ms/step - accuracy: 0.6575 - loss: 1.2514 - val_accuracy: 0.7167 - val_loss: 1.0565 - learning_rate: 1.0000e-04
Epoch 5/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 524s 637ms/step - accuracy: 0.7054 - loss: 1.0796 - val_accura

2025/08/08 03:23:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/08 03:23:13 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


✅ 'sorted_correct_counts.png' has been logged to MLflow.
==================== Full Evaluation Logging Complete ====================



2025/08/08 03:23:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✔️ Run c4a51c5302844401a7e5ea537b444a6f 完成。最佳驗證準確率: 0.8533
🎉 新的最佳模型! Run ID: c4a51c5302844401a7e5ea537b444a6f, Accuracy: 0.8533
🏁 自動調參完成。
🏆 最終最佳模型的 Run ID 為: c4a51c5302844401a7e5ea537b444a6f
🏆 最終最佳驗證準確率為: 0.8533
💾 最佳模型已儲存為 'best_tuned_model.h5'


In [2]:
best_model.save('model_mnV2(best).keras') 

NameError: name 'best_model' is not defined